# Pokemon Battle (fino a 6 pt)
Questo homework sarà un po' diverso dagli altri, perchè non voglio darvi una traccia già definita e/o una soluzione unica perchè voglio che sviluppiate questo progetto con le vostre idee nelle parti che vi interessano di più (o forse sono solo pigro).
L'obiettivo è ampliare e aggiungere funzionalità al progetto della battaglia Pokemon vista a lezione e vi propongo alcune vie che secondo me ha senso percorrere, ma apprezzerò anche idee innovative!
Uniche regole da seguire:
- no programmazione grafica, voglio solo codice a terminale
- nella soluzione scrivete un breve riassunto delle nuove funzionalità che avete aggiunto
- fate quante proposte volete, il punteggio dipenderà sia da qualità che quantità delle vostre aggiunte
- potete modificare il mio codice come volete (e vi suggerisco di farlo), ma cercate di mantenere i meccanismi base
- scrivete codice "bello" (sfruttate le vostre nuove conoscenze!) e leggibile
- assicuratevi che tutto funzioni quando consegnate!

Come potrete vedere, le proposte sono molto sinergiche tra di loro e sicuramente farne mole porta ad un risultato sempre più soddisfacente.
In particolare la 1 e la 2 credo siano le più semplici e sono in grado di dare più valore alle altre se sviluppate con loro.

Se volete la lode sorprendetemi!

## Proposta 1: Creazione Allenatore (1pt)
Create una classe Allenatore tale che abbia una squadra e 'spostate' le funzioni per fare partire una lotta all'interno dell'allenatore. Considerate di potere fare scontri sia contro altri allenatori che pokemon selvaggi (senza allenatore), ed implementate la scelta di cambiare pokemon in una lotta e di fuggire da un pokemon selvaggio.
Infine create una sorta di inventario con pozioni per la salute e pokeball da usare contro i pokemon selvaggi, ovviamente aggiungete anche la possibilità di usare oggetti nelle lotte.

## Proposta 2: Completare la Prima Generazione (1pt)
Create un file con la generazione dei singoli Pokemon del Pokedex della prima generazione e fate in modo che quando volete crearlo nel vostro codice vi basterà digitare il nome ed i dati verranno presi dal file.
Aggiungete anche tutte le immagini dei Pokemon nel file già presente. Per fare questo lavoro dovrete fare un altro script (consegnate anche quello) che vi consenta di riscrivere sul nostro file nel formato che volete i disegni presenti nel sito:
https://gist.github.com/MatheusFaria/4cbb8b6dbe33fd5605cf8b8f7130ba6d

## Proposta 3: Miglioramento Statistiche (2pt)
Create le seguenti statistiche all'interno dei Pokemon: LVL, HP, ATT, DEF, SPD, SPC, TIPO1, TIPO2
Il livello è un numero da 1 a 100 ed influenzerà le 5 stat successive, infatti queste hanno un valore base al lvl 1, diverso per ogni Pokemon ed al lvl 100 gli HP sono il doppio +110 mentre le altre sono il doppio +5; create un incremento lineare sapendo questa informazione e prendete le statistiche base (lvl 1) da qui: https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_base_stats_(Generation_I).
Sistematizzate meglio la gestione dei tipi dei Pokemon e create la tavola che spiega i modificatori di attacco tra i tipi, potete trovarla qui: https://pokemondb.net/type/old (GEN 1)
Infine fate in modo che il Pokemon più veloce attacchi per primo in ogni turno e cambiate la formula dei danni con:

**( ( (LVL * 2/5 +2 ) * ATT * POW )/( 50*DEF ) +2 ) * STAB * TYPEMOD * RANDOM)/255**

POW: Potenza Attacco (se non fate la prossima proposta settate 50 e tutto fisico)
STAB: Vale 1.5 se l'attacco è del tipo del Pokemon che attacca, altrimenti 1
TYPEMOD: Prodotto dei moltiplicatori di tipo (tipo dell'attacco contro entrambi i tipi del difensore)
RANDOM: Numero tra 217 e 255

## Proposta 4: Miglioramento Attacchi (2pt)
Create un file contenete le statistiche di ogni mossa Pokemon, scegliete pure solo gli attacchi e le modifiche di status semplici. Ogni attacco deve avere PP (quante volte si può usare la mossa), POW (forza), ACC (probabilità di riuscita), TIPO e CAT (fisico, speciale o status). Vi ricordo che una mossa speciale coinvolge sia in attacco che in difesa il valore SPC al posto di ATT e DEF, mentre una mossa di status non fa danni.
Create anche i modificatori di stato come ad esempio addormentato, avvelenato, confuso...
Infine sorprendetemi con qualche meccanica complessa!

## Codice Sviluppato a Lezione:



In [1]:
import sys
import random
import time
import math
import os

def delay_print(s):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(0.03)
        
def cls():
    os.system('cls' if os.name=='nt' else 'clear')

In [2]:
#dizionario completo per la prima gen per l'efficacia degli attacchi

attackingDict = {   
                'normal': {'normal': 1.0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 1.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 1.0, 'bug': 1.0, 'rock': 0.5, 'ghost': 0, 'dragon': 1.0},
    
                'fire': {'normal': 1.0, 'fire': 0.5, 'water': 0.5, 'electric': 1.0, 'grass': 2.0, 'ice': 2.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 1.0, 'bug': 2.0, 'rock': 0.5, 'ghost': 1, 'dragon': 0.5},
    
                'water': {'normal': 1.0, 'fire': 2.0, 'water': 0.5, 'electric': 1.0, 'grass': 0.5, 'ice': 1.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 2.0, 'flying': 1.0, 'psychic': 1.0, 'bug': 1.0, 'rock': 2.0, 'ghost': 1, 'dragon': 0.5},
    
                'electric': {'normal': 1.0, 'fire': 1.0, 'water': 2.0, 'electric': 0.5, 'grass': 0.5, 'ice': 1.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 0, 'flying': 2.0, 'psychic': 1.0, 'bug': 1.0, 'rock': 1.0, 'ghost': 1, 'dragon': 0.5},
    
                'grass': {'normal': 1.0, 'fire': 0.5, 'water': 2.0, 'electric': 1.0, 'grass': 0.5, 'ice': 1.0, 'fighting': 1.0, 'poison': 0.5,
                            'ground': 2.0, 'flying': 0.5, 'psychic': 1.0, 'bug': 0.5, 'rock': 2.0, 'ghost': 1, 'dragon': 0.5},
    
                'ice': {'normal': 1.0, 'fire': 1.0, 'water': 0.5, 'electric': 1.0, 'grass': 2.0, 'ice': 0.5, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 2.0, 'flying': 2.0, 'psychic': 1.0, 'bug': 1.0, 'rock': 1.0, 'ghost': 1, 'dragon': 2.0},
    
                'fighting': {'normal': 2.0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 2.0, 'fighting': 1.0, 'poison': 0.5,
                            'ground': 1.0, 'flying': 0.5, 'psychic': 0.5, 'bug': 0.5, 'rock': 2, 'ghost': 0, 'dragon': 1.0},
    
                'poison': {'normal': 1.0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 2.0, 'ice': 1.0, 'fighting': 1.0, 'poison': 0.5,
                            'ground': 0.5, 'flying': 1.0, 'psychic': 1.0, 'bug': 2.0, 'rock': 0.5, 'ghost': 0.5, 'dragon': 1.0},
    
                'ground': {'normal': 1.0, 'fire': 2.0, 'water': 1.0, 'electric': 2.0, 'grass': 0.5, 'ice': 1.0, 'fighting': 1.0, 'poison': 0.5,
                            'ground': 1.0, 'flying': 0, 'psychic': 1.0, 'bug': 0.5, 'rock': 2.0, 'ghost': 1.0, 'dragon': 1.0},
    
                'flying': {'normal': 1.0, 'fire': 1.0, 'water': 1.0, 'electric': 0.5, 'grass': 2.0, 'ice': 1.0, 'fighting': 2.0, 'poison': 1.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 1.0, 'bug': 2.0, 'rock': 0.5, 'ghost': 1.0, 'dragon': 1.0},
    
                'psychic': {'normal': 1.0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 1.0, 'fighting': 2.0, 'poison': 2.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 0.5, 'bug': 1.0, 'rock': 1.0, 'ghost': 1.0, 'dragon': 1.0},
    
                'bug': {'normal': 1.0, 'fire': 0.5, 'water': 1.0, 'electric': 1.0, 'grass': 2.0, 'ice': 1.0, 'fighting': 0.5, 'poison': 2.0,
                            'ground': 1.0, 'flying': 0.5, 'psychic': 2.0, 'bug': 1.0, 'rock': 1.0, 'ghost': 1.0, 'dragon': 1.0},
    
                'rock': {'normal': 1.0, 'fire': 2.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 2.0, 'fighting': 0.5, 'poison': 1.0,
                            'ground': 0.5, 'flying': 2.0, 'psychic': 1.0, 'bug': 2.0, 'rock': 1.0, 'ghost': 1.0, 'dragon': 1.0},
    
                'ghost': {'normal': 0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 1.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 0, 'bug': 1.0, 'rock': 1.0, 'ghost': 2.0, 'dragon': 1.0},
    
                'dragon': {'normal': 1.0, 'fire': 1.0, 'water': 1.0, 'electric': 1.0, 'grass': 1.0, 'ice': 1.0, 'fighting': 1.0, 'poison': 1.0,
                            'ground': 1.0, 'flying': 1.0, 'psychic': 1.0, 'bug': 1.0, 'rock': 1.0, 'ghost': 1.0, 'dragon': 1.0}}

In [ ]:
#dizionario contenente alcuni pokemon della gen 1

dict_pokemon = {
    'nome': ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle', 'Blastoise', 'Caterpie', 'Metapod', 'Butterfree',
            'Weedle', 'Kakuna', 'Beedrill'],
    'stats': [[45, 49, 49, 45, 65], [60, 62, 63, 60, 80], [80, 82, 83, 80, 100], [39, 52, 43, 65, 50], [58, 64, 58, 80, 65], 
              [78, 84, 78, 100, 85], [44, 48, 65, 43, 50], [59, 63, 80, 58, 65], [79, 83, 100, 78, 85], [45, 30, 35, 45, 20], 
              [50, 20, 55, 30, 25], [60, 45, 50, 70, 80], [40, 35, 30, 50, 20], [45, 25, 50, 35, 25], [65, 80, 40, 75, 45]],
    'tipo':['grass', 'grass','grass', 'fire', 'fire', 'fire', 'water', 'water', 'water', 'bug', 'bug', 'bug', 'bug', 'bug', 'bug'], 
    'mosse': [['Frustata', 'Testata', 'Azione', 'Semitraglia'],['Frustata', 'Foglielama', 'Semitraglia', 'Azione'], 
              ['Frustata', 'Foglielama', 'Terremoto', 'Radicalbero'], ['Braciere', 'Graffio', 'Azione', 'Fuocopugno'],
              ['Braciere', 'Graffio', 'Lanciafiamme', 'Fuocopugno'], ['Lanciafiamme', 'Volo', 'Incendio', 'Fuocopugno'], 
              ['Azione', 'Pistolacqua', 'Morso', 'Idropulsar'], ['Idropulsar', 'Idrondata', 'Surf', 'Morso'], 
              ['Surf', 'Cascata', 'Idropompa', 'Idrondata'], ['Azione', 'Coleomorso', 'Elettrotela'], 
              ['Azione', 'Coleomorso', 'Elettrotela'], ['Eterelama', 'Ronzio', 'Confusione', 'Retormarcia'], 
              ['Azione', 'Velenospina', 'Coleomorso'], ['Azione', 'Velenospina', 'Coleomorso', 'Elettrotela'], 
              ['Pungiglione', 'Velenpuntura', 'Missilspillo', 'Velenoshock']],
    'stab': [[1,0,0,1], [1,1,1,0], [1,1,0,1], [1,0,0,1], [1,0,1,1], [1,0,1,1], [0,1,0,1], [1,1,1,0], 
             [1,1,1,1], [0,1,0], [0,1,0], [1,1,0,1], [0,0,1], [0,0,1,0], [1,1,1,1]]
}

In [ ]:
class Trainer():
    
    '''Classe per creare un allenatore pokemon con pokemon casuali'''
    def __init__(self):
            self.squadra = []

    def creaSquadra(self):
        for _ in range(2):
            pkmn_selection = random.randint(1, 15)
            pkmn_caratt = [dict_pokemon['nome'][pkmn_selection], dict_pokemon['stats'][pkmn_selection], 
                                      dict_pokemon['tipo'][pkmn_selection], dict_pokemon['mosse'][pkmn_selection],
                                      dict_pokemon['stab'][pkmn_selection]]
            self.squadra.append(Pokemon(pkmn_caratt[0], pkmn_caratt[1], pkmn_caratt[2], pkmn_caratt[3], pkmn_caratt[4]))


    def creaBorsa(self):
        self.pozione = 5
        self.superpozione = 5
        self.iperpozione = 5
        self.curatotale = 5
        self.pokeball = 5
        self.megaball = 5
        self.ultraball = 5       

In [ ]:
def menuChoice(): 
        delay_print('\n1 - Lotta')
        delay_print('\n2 - Pokemon')
        delay_print('\n3 - Borsa')
        delay_print('\n4 - Fuga')
        choiceStr = input("Scegli un'opzione: ")
        while(choiceStr!="1" and choiceStr!="2" and choiceStr!="3" and choiceStr!="4"):
            choiceStr = input("Scegli un'opzione valida: ")
        return int(choiceStr)
        
def attack(trainer1, trainer2, attackCode):
        attackName = trainer1.squadra[0].attacks[attackCode]
        
        attackMod = 1
        stab = 1
        if(trainer1.squadra[0].typeattack[attackCode]):
            attackMod = attackingDict[trainer1.squadra[0].type][trainer2.squadra[0].type]
        valRandAtk = random.randint(217, 255)
        if trainer1.squadra[0].typeattack[attackCode] == 1:
            stab = 1.5
        attackDamage = math.ceil((((trainer1.squadra[0].level * 2/5 +2) * trainer1.squadra[0].atk * 50)/(50 * trainer2.squadra[0].defense) + 2) * stab * attackMod * valRandAtk / 255)   
            
        strDamage = ""
        if(attackMod < 1):
            strDamage += "\nNon è molto efficace..."
        elif(attackMod > 1):
            strDamage += "\nE' superefficace!"
            
        return attackName, attackDamage, strDamage
        
        
def attackChoice(trainer1, you, humanPlayer):
        if(you and humanPlayer):
            for i, x in enumerate(trainer1.squadra[0].attacks):
                delay_print(f"\n{i+1}. {x}")
            delay_print('\n5. Torna indietro')
            attackStr = input('\nScegli una mossa: ')
            while(attackStr!="1" and attackStr!="2" and attackStr!="3" and attackStr!="4" and attackStr != "5"):
                attackStr = input('\nScegli una mossa valida: ')
            return int(attackStr)-1
        else:
            return random.randint(0,len(trainer1.squadra[0].attacks)-1)

        
def scambiaPokemon(trainer, counter):
    trainer.squadra[0], trainer.squadra[counter -1] = trainer.squadra[counter -1], trainer.squadra[0]
    return trainer
    
    
def sceltaBorsa(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp):
    
    delay_print(f"\n1. Pozione - {trainer1.pozione} rimanenti")
    delay_print(f"\n2. Superpozione - {trainer1.superpozione} rimanenti")
    delay_print(f"\n3. Iperpozione - {trainer1.iperpozione} rimanenti")
    delay_print(f"\n4. Cura totale - {trainer1.curatotale} rimanenti")
    delay_print("\n5. Torna indietro")
    selezione_borsa = input("Seleziona strumento: ")
                
    while(selezione_borsa!="1" and selezione_borsa!="2" and selezione_borsa!="3" and selezione_borsa!="4" and selezione_borsa!="5"):
        selezione_borsa = input('Scegli un opzione valida: ')
    if selezione_borsa == "1":
        if trainer1.pozione > 0:
            trainer1.pozione -= 1
            return int(20)
        elif trainer1.pozione <= 0:
            delay_print("Pozioni finite, selezionare un'altro strumento!")
            return sceltaBorsa(trainer1)
    elif selezione_borsa == "2":
        if trainer1.superpozione > 0:
            trainer1.superpozione -= 1
            return int(50)
        elif trainer1.superpozione <= 0:
            delay_print("Superpozioni finite, selezionare un'altro strumento!")
            return sceltaBorsa(trainer1)
    elif selezione_borsa == "3":
        if trainer1.iperpozione > 0:
            trainer1.iperpozione -= 1
            return int(200) #nella gen 1 curavano 200 hp, dannata gen7
        elif trainer1.iperpozione <= 0:
            delay_print("Iperpozioni finite, selezionare un'altro strumento!")
            return sceltaBorsa(trainer1)
    elif selezione_borsa == "4":
        if trainer1.curatotale > 0:
            trainer1.curatotale -= 1
            return int(3000)
        elif trainer1.curatotale <= 0:
            delay_print("Cure totali finite, selezionare un'altro strumento!")
            return sceltaBorsa(trainer1)
    elif selezione_borsa == "5":
        return battle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
    
def scegliPokemon(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp):
    
    for i in range(len(trainer1.squadra)):
        delay_print(f"\n{i+1}. {trainer1.squadra[i].name} L.{trainer1.squadra[i].level} - {max(0, trainer1.squadra[i].HP)} HP")
    delay_print(f"\n{len(trainer1.squadra) + 1}. Torna indietro")
    
    prob1 = True
    prob2 = True
    
    while(prob1 == True or (prob1 == False and prob2 == True)):
        selezione_pokemon = input("Seleziona un pokemon")
        if(int(selezione_pokemon) > len(trainer1.squadra)+1 or int(selezione_pokemon) <= 1):
            prob1 = True
        elif int(selezione_pokemon) == len(trainer1.squadra)+1:
            return battle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
        else:
            prob1 = False
        if(prob1 == False):
            if(trainer1.squadra[int(selezione_pokemon)-1].HP <= 0):
                prob2 = True
            else:
                prob2 = False
        if(prob1 == True and prob2 == True):
            delay_print("\nOpzione non valida!")
            
    return int(selezione_pokemon)
    

def battle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp):   
        
        if you:
            
            if(trainer1.squadra[0].HP <= 0 and counterYou > 0):
                counterYou -= 1
                trainer1 = scambiaPokemon(trainer1, counterYou)
                return startBattle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
            
            elif(trainer1.squadra[0].HP <= 0 and counterYou <= 0):
                    money = random.randint(100,5000)
                    delay_print(f"\nNon hai piu' pokemon disponibili!")
                    delay_print("\nL'avversario ha vinto!)")
                    delay_print(f"\nDai all'avversario ${money}.\n")
                    return trainer2, trainer1
            else:
                scelta_menu = menuChoice()
                if scelta_menu == 1:

                    if(trainer1.squadra[0].HP > 0):
                        attackCode = attackChoice(trainer1, you, humanPlayer)
                        if attackCode == 4:
                            return battle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
                        else:
                            attackName, attackDamage, strDamage = attack(trainer1, trainer2, attackCode)
                            trainer2.squadra[0].HP -= attackDamage 

                            delay_print(f"\n{trainer1.squadra[0].name} usa {attackName}")
                            delay_print(strDamage)
                            delay_print(f"\n{trainer2.squadra[0].name} ha {max(trainer2.squadra[0].HP,0)} HP rimasti\n")

                            return battle(trainer2, trainer1, not(you), humanPlayer, pokemonArt, counterOpp, counterYou)

                elif scelta_menu == 2:
                    index_scambio = scegliPokemon(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
                    trainer1 = scambiaPokemon(trainer1, index_scambio)
                    
                    if(pokemonArt):
                        cls()
                        print(trainer1.squadra[0])
                        delay_print("--- VERSUS ---\n")
                        print(trainer2.squadra[0])
                    else:
                        cls()
                        delay_print(f"\n{trainer1.squadra[0].name} L.{trainer1.squadra[0].level} - {trainer1.squadra[0].HP}HP\t VS\t {trainer2.squadra[0].name} L.{trainer2.squadra[0].level} - {trainer2.squadra[0].HP}HP\n")
         
                    return battle(trainer2, trainer1, not(you), humanPlayer, pokemonArt, counterOpp, counterYou) 
                    
                elif scelta_menu == 3:
                    hp_curati = sceltaBorsa(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
                    delay_print(f"\n{trainer1.squadra[0].name} ha ripristinato {hp_curati} hp")

                    trainer1.squadra[0].HP = min(trainer1.squadra[0].maxHP, trainer1.squadra[0].HP + hp_curati)
                    return battle(trainer2, trainer1, not(you), humanPlayer, pokemonArt, counterOpp, counterYou)


                elif scelta_menu == 4:
                    delay_print("Non puoi fuggire durante una battaglia!")
                    return battle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)

        else:
            
            if(trainer1.squadra[0].HP <= 0 and counterYou > 0):
                counterYou -= 1
                trainer1 = scambiaPokemon(trainer1, counterYou)
                return startBattle(trainer2, trainer1, not(you), humanPlayer, pokemonArt, counterOpp, counterYou)
            
            elif(trainer1.squadra[0].HP <= 0 and counterYou <= 0):
                money = random.randint(100,5000)
                delay_print(f"\nL'avversario non ha piu' pokemon disponibili!")
                delay_print("\nHai vinto!)")
                delay_print(f"\nL'avversario ti da ${money}.\n")
                
                return trainer1, trainer2
            
            elif(trainer1.squadra[0].HP > 0):
                attackCode = attackChoice(trainer1, you, humanPlayer)
                attackName, attackDamage, strDamage = attack(trainer1, trainer2, attackCode)
                trainer2.squadra[0].HP -= attackDamage 

                delay_print(f"\n{trainer1.squadra[0].name} usa {attackName}")
                delay_print(strDamage)
                delay_print(f"\n{trainer2.squadra[0].name} ha {max(trainer2.squadra[0].HP,0)} HP rimasti\n")

                return battle(trainer2, trainer1, not(you), humanPlayer, pokemonArt, counterOpp, counterYou)
            
        return trainer1, trainer2


def startBattle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp):
        if(pokemonArt):
            cls()
            print(trainer1.squadra[0])
            delay_print("--- VERSUS ---\n")
            print(trainer2.squadra[0])
        else:
            cls()
            delay_print(f"\n{trainer1.squadra[0].name} L.{trainer1.squadra[0].level} - {trainer1.squadra[0].HP}HP\t VS\t {trainer2.squadra[0].name} L.{trainer2.squadra[0].level} - {trainer2.squadra[0].HP}HP\n")
         
        return battle(trainer1, trainer2, you, humanPlayer,pokemonArt, counterYou, counterOpp)


def main(you, humanPlayer, pokemonArt):
    
    trainer1 = Trainer()
    trainer1.creaSquadra()
    trainer1.creaBorsa()
    trainer2 = Trainer()
    trainer2.creaSquadra()
    trainer2.creaBorsa()
        
    counterYou = len(trainer1.squadra)
    counterOpp = len(trainer2.squadra)
        
    winner, loser = startBattle(trainer1, trainer2, you, humanPlayer, pokemonArt, counterYou, counterOpp)
       

In [ ]:
class Pokemon(object):
    
    '''Classe per creare un pokemon'''
    
    def __init__(self, name, stats, type, attacks, typeattack):    #stats = [HP, atk, defense, speed, special]
        self.name = name
        self.level = random.randint(5, 100)
        self.HP = math.ceil(((stats[0] + 110)/100) * self.level + stats[0])
        self.atk = math.ceil(((stats[1] + 5)/100) * self.level + stats[1])
        self.defense = math.ceil(((stats[2] + 110)/100) * self.level + stats[2])
        self.speed = math.ceil(((stats[3] + 110)/100) * self.level + stats[3])
        self.special = math.ceil(((stats[4] + 110)/100) * self.level + stats[4])
        self.type = type
        self.attacks = attacks
        self.typeattack = typeattack
        self.maxHP = math.ceil(((stats[0] + 110)/100) * self.level + stats[0])
        
        
    def __str__(self):
        fileName = "PokemonArt"
        image = R""" """
        image += f"""{self.name} L.{self.level} - {self.HP}HP\n"""
        file = open(f"{fileName}.txt", "r")
        text = file.readlines()
        add = False
        for line in text:
            if(line.strip() == self.name):
                add = True
            elif(add):
                if(line.strip() == "STOP"):
                    return image
                else:
                    image += line.rstrip()
                    image += """\n"""
        return "Immagine non Trovata"

In [ ]:
main(True,True, True)